In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_309982/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/14 14:37:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/14 14:37:20 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [4]:
import pandas as pd

# Sample data for Players table
players_data = {
    'player_id': [15, 25, 30, 45, 10, 35, 50, 20, 40],
    'group_id': [1, 1, 1, 1, 2, 2, 2, 3, 3]
}

# Sample data for Matches table
matches_data = {
    'match_id': [1, 2, 3, 4, 5],
    'first_player': [15, 30, 30, 40, 35],
    'second_player': [45, 25, 15, 20, 50],
    'first_score': [3, 1, 2, 5, 1],
    'second_score': [0, 2, 0, 2, 1]
}

# Create DataFrame for Players and Matches
players_df = pd.DataFrame(players_data)
matches_df = pd.DataFrame(matches_data)



print(players_df)
print()
print(matches_df)
print()




df_person = spark.createDataFrame(players_df)
df_person.createOrReplaceTempView("Players")

df_person = spark.createDataFrame(matches_df)
df_person.createOrReplaceTempView("Matches")



   player_id  group_id
0         15         1
1         25         1
2         30         1
3         45         1
4         10         2
5         35         2
6         50         2
7         20         3
8         40         3

   match_id  first_player  second_player  first_score  second_score
0         1            15             45            3             0
1         2            30             25            1             2
2         3            30             15            2             0
3         4            40             20            5             2
4         5            35             50            1             1



In [19]:
query = """
    with first_players as (
        select
            first_player as player_id,
            first_score as score
        from Matches m
    ),
    
    second_players as (
        select
            second_player as player_id,
            second_score as score
        from Matches m
    ),
    
    all_players as (
        select
            t.player_id,
            sum(t.score) as score,
            p.group_id
        from (
            select * from first_players
            union all
            select * from second_players
        ) t left join Players p on t.player_id = p.player_id
        group by t.player_id, p.group_id
    )
    
    select 
        group_id,
        player_id
    from (
        select
            player_id,
            score,
            group_id,
            dense_rank() over (partition by group_id order by score desc, player_id asc) as rk
        from all_players a
    ) where rk = 1
"""

In [20]:
spark.sql(query).show()

+--------+---------+
|group_id|player_id|
+--------+---------+
|       1|       15|
|       2|       35|
|       3|       40|
+--------+---------+

